In [ ]:
# Use secrets DBUtil to get Snowflake credentials.
user = dbutils.secrets.get(scope="key-vault-secret-scope",key="snowflake-username")
password = dbutils.secrets.get(scope="key-vault-secret-scope",key="snowflake-password")
url = dbutils.secrets.get(scope="key-vault-secret-scope",key="snowflake-url")

# snowflake connection options
options = {
  "sfUrl": url,
  "sfUser": user,
  "sfPassword": password,
  "sfDatabase": "SHARED_CRYPTO_QUOTES",
  "sfSchema": "PUBLIC",
  "sfWarehouse": "READER_WH"
}

In [ ]:
df = spark.read.format("snowflake").options(**options).option("query", "CRYPTO_QUOTES").load()

display(df)

In [ ]:
print(df.count())

In [ ]:
query = "SELECT * \
         FROM SHARED_CRYPTO_QUOTES.PUBLIC.CRYPTO_QUOTES \
         WHERE PRICE_DATE > '2022-04-06'"

df = spark.read.format("snowflake").options(**options).option("query", query).load()

display(df)

In [ ]:
print(df.count())

In [ ]:
spark.conf.set(
    "fs.azure.account.key.cryptoanalyticslake.dfs.core.windows.net",
    dbutils.secrets.get(scope="key-vault-secret-scope",key="cryptoanalyticslake-access-key"))

In [ ]:
destinationfolderpath = f"abfss://crypto-bronze@cryptoanalyticslake.dfs.core.windows.net/snowflake-raw/databricks"

print(destinationfolderpath)

df.write.partitionBy("PRICE_DATE").format("delta").mode("overwrite").save(destinationfolderpath)